# SOUTH CAROLINA POLICE SHOOTING DATA

Code to import necessary dependencies 

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.models.markers import Diamond

import os
import sqlite3
import csv
import datetime
import math

import pandas as pd
import numpy as np

#read the CSV
df = pd.read_csv('compiled_data.csv')

#connect to a database
conn = sqlite3.connect("policeData.db") #if the db does not exist, this creates a policeData.db file in the current directory

#store table in the database:
df.to_sql('data', conn,  if_exists='replace')

output_notebook()



Loading BokehJS ...

## Clean up some of the data
Convert columns to datetime and numbers

In [2]:
#converts opened and time columns to datatime
df[['opened', 'time']] = df[['opened', 'time']].apply(pd.to_datetime)

#replaces NaN with 0
df[['officersInjured', 'officersKilled', 'suspectsInjured', 'suspectsKilled']] = df[['officersInjured', 'officersKilled', 'suspectsInjured', 'suspectsKilled']].fillna(0)

#replaces empty cells with 0
df['officersInjured'].replace(' ', 0, inplace=True)
df['officersKilled'].replace(' ', 0, inplace=True)
df['suspectsInjured'].replace(' ', 0, inplace=True)
df['suspectsKilled'].replace(' ', 0, inplace=True)

#converts columns to int
df[['officersInjured', 'officersKilled', 'suspectsInjured', 'suspectsKilled']] = df[['officersInjured', 'officersKilled', 'suspectsInjured', 'suspectsKilled']].astype(int)


County names

In [3]:
#creates consistent county names
df['county'].replace('(ORANGEBURG)', 'ORANGEBURG', inplace=True)
df['county'].replace(['YORK ', 'YORK *'], 'YORK', inplace=True)
df['county'] = df['county'].str.upper()

Agency names

In [4]:
#creates consistent agency names
df['agency'] = df['agency'].fillna('unknown')
df['agency'].replace(to_replace=' COUNTY ', value=' CO ', regex=True, inplace=True)
df['agency'].replace(to_replace=' METH COL ', value=' Methodist College ', regex=True, inplace=True)
df['agency'].replace(to_replace=' PD', value=' police department', regex=True, inplace=True)
df['agency'].replace(to_replace=' PS', value=' public safety', regex=True, inplace=True)
df['agency'].replace(to_replace=' PD ', value=' police department ', regex=True, inplace=True)
df['agency'].replace(to_replace=' SO', value=' county sheriff office', regex=True, inplace=True)
df['agency'].replace(to_replace=' CO', value=' county', regex=True, inplace=True)
df['agency'].replace(to_replace=' CO ', value=' county ', regex=True, inplace=True)
df['agency'].replace(to_replace='PPP', value='proabtion parole and pardon services', regex=True, inplace=True)
df['agency'].replace(to_replace='SCDC', value='South Carolina Department of Corrections', regex=True, inplace=True)
df['agency'].replace(to_replace='USMS', value='U.S. Marshals Service', regex=True, inplace=True)
df['agency'].replace(to_replace=' DPS ', value=' department of public safety ', regex=True, inplace=True)
df['agency'].replace(to_replace=' DPS', value=' department of public safety', regex=True, inplace=True)
df['agency'].replace(to_replace=' DEPT. PUB. ', value=' department of public safety ', regex=True, inplace=True)
df['agency'].replace(to_replace=' DEPT. ', value=' department ', regex=True, inplace=True)
df['agency'].replace(to_replace='DNR', value='department of natural resources', regex=True, inplace=True)
df['agency'].replace(to_replace='MT ', value='MT. ', regex=True, inplace=True)
df['agency'].replace(to_replace='N. ', value='NORTH ', regex=True, inplace=True)
df['agency'].replace(to_replace='US ', value='U.S. ', regex=True, inplace=True)
df['agency'].replace(to_replace='US ', value='U.S. ', regex=True, inplace=True)
df['agency'] = df['agency'].str.upper()
df['agency'].replace(to_replace=' COUNTY COUNTY ', value=' COUNTY ', regex=True, inplace=True)
df['agency'].replace(to_replace='BERKELEY COUNTY SHERIFF OFFICE ', value='BERKELEY COUNTY SHERIFF OFFICE', regex=True, inplace=True)
df['agency'].replace(to_replace='YORK COUNTY SHERIFF OFFICE ', value='YORK COUNTY SHERIFF OFFICE', regex=True, inplace=True)
df['agency'].replace(to_replace='COUNTYRRECTIONS', value='CORRECTIONS', regex=True, inplace=True)
df['agency'].replace(to_replace='SCHP', value='SOUTH CAROLINA HIGHWAY PATROL', regex=True, inplace=True)
df['agency'].replace('(DEA *****)', 'DEA', inplace=True)
df['agency'].replace('SLED - S/A AARON RAWL', 'SLED', regex=True, inplace=True)

Offense names

In [5]:
#creates consistent offense names
df['offense'].replace(to_replace='unk', value='UNKNOWN', regex=True, inplace=True)
df['offense'] = df['offense'].fillna('unknown')
df['offense'] = df['offense'].str.upper()
df['offense'].replace(to_replace='DOMESTIC CALL', value='DOMESTIC', regex=True, inplace=True)
df['offense'].replace(to_replace=' NARCOTICS WARRANT', value='NARCOTICS WARRANT', regex=True, inplace=True)
df['offense'].replace(to_replace='VEHICLE B & E', value='BREAKING & ENTERING (AUTO)', regex=True, inplace=True)
df['offense'].replace(to_replace=['B & E', 'BREAK-IN'], value='BREAKING & ENTERING', regex=True, inplace=True)
df['offense'].replace(to_replace='BARRICATED', value='BARRICADED', regex=True, inplace=True)
df['offense'].replace(to_replace='BURGLARY SUSPECT', value='BURGLARY', regex=True, inplace=True)
df['offense'].replace(to_replace=['DISTURBANCE CALL', 'DISTURBANCE/FIGHT'], value='DISTURBANCE', regex=True, inplace=True)
df['offense'].replace(to_replace='911 HANG UP CALL / OFF RESPONDED', value='911 CALL', regex=True, inplace=True)
df['offense'].replace(to_replace='ALARM CALL/HOME INVASION', value='HOME INVASION', regex=True, inplace=True)
df['offense'].replace(to_replace=['CAR CHASE', 'VEHICLE CHASE', 'SHOPLIFTING/VEHICLE PURSUIT'], value='VEHICLE PURSUIT', regex=True, inplace=True)
df['offense'].replace(to_replace=['SHOTS FIRED CALL', 'SHOOTING COMPLAINT', 'DISTURBANCE/SHOTS FIRED'], value='SHOTS FIRED', regex=True, inplace=True)
df['offense'].replace(to_replace=['SUICIDAL SUBJECT', 'SUICIDE CALL'], value='SUICIDE THREAT', regex=True, inplace=True)
df['offense'].replace(to_replace=['SUSPICIOUS PERSON', 'SUSPICIOUS VEHICLE', 'SUSPECIOUS VEHICLE'], value='SUSPICIOUS ACTIVITY', regex=True, inplace=True)
df['offense'].replace(to_replace=['MENTAL HEALTH PATIENT', 'MENTAL HEALTH TRANSPORT', 'MENTAL PATIENT', 'MENTAL SUBJECT', 'MENTAL SUBJECT/WELFARE CHECK', 'MENTAL HEALTH ISSUES/WELFARE CHECK'], value='MENTAL HEALTH ISSUES', regex=True, inplace=True)
df['offense'].replace(to_replace=['ARREST WARRANT SERVICE', 'BENCH WARRANT', 'DRUG WARRANT ', 'FUGITIVE WARRANT', 'NARCOTICS WARRANT', 'SERVING WARRANT', 'WARRANT SERVICESS'], value='WARRANT SERVICES', regex=True, inplace=True)
df['offense'].replace(to_replace='REPORT OF POSSIBLE ARMED ROBBERY', value='ARMED ROBBERY', regex=True, inplace=True)
df['offense'].replace(to_replace='SUSPECT STEALING CALL', value='THEFT', regex=True, inplace=True)
df['offense'].replace(to_replace=['DRUG COMPLAINT', 'DRUG OPERATION', 'NARCOTICS INVESTIGATION'], value='DRUG INVESTIGATION', regex=True, inplace=True)
df['offense'].replace(to_replace='SHOPLIFTING SUSPECT', value='SHOPLIFTING', regex=True, inplace=True)
df['offense'].replace(to_replace=['TRAFFIC STOP/DRUGS', 'TRAFFIC STOP/STOLEN VEH', 'NARCOTICS/TRAFFIC STOP' ], value='TRAFFIC STOP', regex=True, inplace=True)
df['offense'].replace(to_replace='GANG MEMBER CONFRONT', value='GANG MEMBER CONFRONTATION', regex=True, inplace=True)
df['offense'].replace(to_replace='THREATENING SOMEONE WITH GUN', value='MAN WITH A GUN', regex=True, inplace=True)
df['offense'] = df['offense'].str.upper()

Officer and suspect ethnicity 

In [6]:
#replaces officerEthnicity data with readable values
df['officerEthnicity'] = df['officerEthnicity'].fillna('unknown')
df['officerEthnicity'].replace('B', 'black', inplace=True)
df['officerEthnicity'].replace('BL', 'black & latino', inplace=True)
df['officerEthnicity'].replace(to_replace=['BW', 'BW ','WB'], value='white & black', inplace=True)
df['officerEthnicity'].replace('L', 'latino', inplace=True)
df['officerEthnicity'].replace(to_replace=['LW', 'WL'], value='white & latino', inplace=True)
df['officerEthnicity'].replace(to_replace=['W', 'W '], value='white', inplace=True)
df['officerEthnicity'] = df['officerEthnicity'].str.upper()

#replaces suspectEthnicity data with readable values
df['suspectEthnicity'] = df['suspectEthnicity'].fillna('unknown')
df['suspectEthnicity'].replace('B', 'black', inplace=True)
df['suspectEthnicity'].replace('BL', 'black & latino', inplace=True)
df['suspectEthnicity'].replace(to_replace=['BW', 'BW ','WB'], value='white & black', inplace=True)
df['suspectEthnicity'].replace('L', 'latino', inplace=True)
df['suspectEthnicity'].replace(to_replace=['LW', 'WL'], value='white & latino', inplace=True)
df['suspectEthnicity'].replace(to_replace=['W', 'W '], value='white', inplace=True)
df['suspectEthnicity'].replace('I', 'indian', inplace=True)
df['suspectEthnicity'] = df['suspectEthnicity'].str.upper()



## SQL selections
This data set originates from the Greenville News, so we should look at look at the number of suspects killed or injured by the local law enforcement agencies (Greenville Police Department and Greenville County Sheriff's Office).

In [7]:
# Here are the SQL selections I need to fullfill the project requirement

pd.read_sql_query('''SELECT agency as Agency, SUM(suspectsKilled) as 'Suspects Killed' 
                    FROM data 
                    WHERE agency LIKE 'greenville%'
                    GROUP BY agency 
                    ORDER BY [Suspects Killed] DESC''', conn)


,Agency,Suspects Killed
0,GREENVILLE COUNTY SO,10.0
1,GREENVILLE CO SO,7.0
2,GREENVILLE PD,2.0
3,GREENVILLE PD & SO,1.0
4,GREENVILLE SO,1.0
5,GREENVILLE PD,0.0


In [8]:
# Another SQL selections I need to fullfill the project requirement

pd.read_sql_query('''SELECT agency as Agency, SUM(suspectsInjured) as 'Suspects Injured' 
                    FROM data 
                    WHERE agency LIKE 'greenville%'
                    GROUP BY agency 
                    ORDER BY [Suspects Injured] DESC''', conn)

,Agency,Suspects Injured
0,GREENVILLE CO SO,9.0
1,GREENVILLE PD,2.0
2,GREENVILLE SO,2.0
3,GREENVILLE PD,1.0
4,GREENVILLE PD & SO,0.0
5,GREENVILLE COUNTY SO,NaN


The Greenville County Sherrif's Office has a higher number of incidents than other local law enforcement agencies. We should examine some of the data surround those incidents

In [9]:
# Offenses the Greenville County Sherrif's Office was responding to where a suspect was killed or injured
  
pd.read_sql_query('''SELECT offense as Offense, SUM(suspectsKilled) as 'Suspects Killed', SUM(suspectsInjured) as 'Suspects Injured'
                    FROM data 
                    WHERE agency LIKE 'greenville co%'
                    GROUP BY offense 
                    ORDER BY [Suspects Killed] DESC''', conn)

,Offense,Suspects Killed,Suspects Injured
0,ALARM CALL,1.0,1.0
1,ALARM CALL/HOME INVASION,1.0,NaN
2,ARMED ROBBERY,1.0,2.0
3,ARMED SUBJECT,1.0,NaN
4,DISTURBANCE,1.0,NaN
5,DISTURBANCE CALL,1.0,1.0
6,DOMESTIC,1.0,3.0
7,DOMESTIC KIDNAPPING,1.0,NaN
8,MAN WITH A GUN,1.0,NaN
9,Report of possible ARMED ROBBERY,1.0,NaN


In [10]:
# Suspect's ethnicity who were killed Greenville County Sherrif's Office 
  
pd.read_sql_query('''SELECT suspectEthnicity as 'Suspect Ethnicity', SUM(suspectsKilled) as 'Suspects Killed'
                    FROM data 
                    WHERE agency LIKE 'greenville co%'
                    GROUP BY [Suspect Ethnicity]
                    ORDER BY [Suspect Ethnicity] DESC''', conn)


,Suspect Ethnicity,Suspects Killed
0,W,7.0
1,L,1.0
2,B,5.0
3,None,4.0


## Data Visualizations
First a look at the new data frame showing all the data in the columns that were changed in the data clean up

In [11]:
df[[
    'opened', 
    'county', 
    'agency', 
    'officersInjured', 
    'officersKilled', 
    'suspectsInjured', 
    'suspectsKilled', 
    'offense', 
    'officerEthnicity', 
    'suspectEthnicity']]
    

,opened,county,agency,officersInjured,officersKilled,suspectsInjured,suspectsKilled,offense,officerEthnicity,suspectEthnicity
0,2018-01-03,DORCHESTER,NORTH CHARLESTON POLICE DEPARTMENT,0,0,1,0,ARMED ROBBERY,UNKNOWN,UNKNOWN
1,2018-01-11,LEXINGTON,LEXINGTON COUNTY SHERIFF OFFICE,0,0,0,1,TRAFFIC STOP,UNKNOWN,UNKNOWN
2,2018-01-16,YORK,YORK COUNTY SHERIFF OFFICE/ YORK POLICE DEPART...,3,1,1,0,DOMESTIC,UNKNOWN,UNKNOWN
3,2018-01-21,SPARTANBURG,LAURENORTH COUNTY SHERIFF OFFICE,0,0,1,0,STOLEN VEHICLE,UNKNOWN,UNKNOWN
4,2018-01-24,SPARTANBURG,GREER POLICE DEPARTMENT,1,0,1,0,SUSPICIOUS ACTIVITY,UNKNOWN,UNKNOWN
5,2018-02-15,ANDERSON,ANDERSON COUNTY SHERIFF OFFICE,0,0,1,0,TRAFFIC STOP,UNKNOWN,UNKNOWN
6,2018-02-27,CHESTER,CHESTER COUNTY SHERIFF OFFICE / CHESTER POLICE...,0,0,0,1,STOLEN VEHICLE,UNKNOWN,UNKNOWN
7,2018-03-02,BERKELEY,BERKELEY COUNTY SHERIFF OFFICE,1,0,1,0,DOMESTIC,UNKNOWN,UNKNOWN
8,2018-03-15,AIKEN,AIKEN COUNTY SHERIFF OFFICE,0,0,1,0,MENTAL HEALTH ISSUES,UNKNOWN,UNKNOWN
9,2018-03-19,GREENVILLE,GREENVILLE COUNTY SHERIFF OFFICE,0,0,0,1,SUICIDE THREAT,UNKNOWN,UNKNOWN


Suspects  killed or injured by the county

In [12]:
for ind, row in df.iterrows():
    df.loc[ind, "suspectsTotal"] = row['suspectsKilled'] + row['suspectsInjured']

#create a suspects killed by county variable that returns a new data instead of a groupby object
suspects_df = df.groupby(['county'])['suspectsTotal'].sum().reset_index()

#removes any county that did not kill a suspect
suspects_df = suspects_df[suspects_df.suspectsTotal > 0]

#sort the number killed from highest to lowest
suspects_df = suspects_df.sort_values(['suspectsTotal'], ascending=False)

#create a list of unique county names
co = list(suspects_df.county.unique())

#creates data source from the new data frame
s_source = ColumnDataSource(suspects_df)

#create hover tools
hover = HoverTool()
hover.tooltips = """
    <div>
        <h3>@county COUNTY</h3>
        <div><strong>Suspects killed or injured:</strong> @suspectsTotal</div>
    </div>
"""

#add plot
p = figure(
    x_range=co,
    plot_height=600,
    plot_width=1000,
    title="Suspects killed or injured by county",
    x_axis_label="County",
    y_axis_label="Suspects Killed or Injured",
    toolbar_location=None,
    tools="")

p.vbar(
    x='county',
    top='suspectsTotal',
    width=0.9,
    color='#009bff',
    #fill_alpha=0.9,
    source=s_source)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/3
p.add_tools(hover)

#show the result
show(p)


In [13]:
for ind, row in df.iterrows():
    df.loc[ind, "suspectsTotal"] = row['suspectsKilled'] + row['suspectsInjured']

#create a suspects injured by county variable that returns a new data instead of a groupby object
suspects_df = df.groupby(['offense'])['suspectsTotal'].sum().reset_index()

#removes any county that did not injure a suspect
suspects_df = suspects_df[suspects_df.suspectsTotal > 0]

#sort the number injured from highest to lowest
suspects_df = suspects_df.sort_values(['suspectsTotal'], ascending=False)

#create a list of unique county names
of = list(suspects_df.offense.unique())

#creates data source from the new data frame
s_source = ColumnDataSource(suspects_df)

#create hover tools
hover = HoverTool()
hover.tooltips = """
    <div>
        <h3>@offense</h3>
        <div><strong>Suspects injured or killed:</strong> @suspectsTotal</div>
    </div>
"""

#add plot
p = figure(
    x_range=of,
    plot_height=600,
    plot_width=1000,
    title="Suspects injured or killed by offense",
    x_axis_label="offense",
    y_axis_label="Suspects",
    toolbar_location=None,
    tools="")

p.vbar(
    x='offense',
    top='suspectsTotal',
    width=0.9,
    color='#009bff',
    #fill_alpha=0.9,
    source=s_source)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/3
p.add_tools(hover)

#show the result
show(p)


In [17]:
for ind, row in df.iterrows():
    df.loc[ind, "suspectsTotal"] = row['suspectsKilled'] + row['suspectsInjured']

#create a suspects injured by county variable that returns a new data instead of a groupby object
suspects_df = df.groupby(['suspectEthnicity'])['suspectsTotal'].sum().reset_index()

#removes any county that did not injure a suspect
suspects_df = suspects_df[suspects_df.suspectsTotal > 0]

#sort the number injured from highest to lowest
suspects_df = suspects_df.sort_values(['suspectsTotal'], ascending=False)

#create a list of unique county names
se = list(suspects_df.suspectEthnicity.unique())

#creates data source from the new data frame
s_source = ColumnDataSource(suspects_df)

#create hover tools
hover = HoverTool()
hover.tooltips = """
    <div>
        <h3>@suspectEthnicity</h3>
        <div><strong>Suspects injured or killed:</strong> @suspectsTotal</div>
    </div>
"""

#add plot
p = figure(
    x_range=se,
    plot_height=400,
    plot_width=800,
    title="Ethnicity of suspects injured or killed",
    x_axis_label="Ethnicity",
    y_axis_label="Suspects",
    toolbar_location=None,
    tools="")

p.vbar(
    x='suspectEthnicity',
    top='suspectsTotal',
    width=0.9,
    color='#009bff',
    #fill_alpha=0.9,
    source=s_source)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.add_tools(hover)

#show the result
show(p)


#### Look at the number of suspects and officers killed since 2009

In [18]:
for ind, row in df.iterrows():
    df.loc[ind, "suspectsTotal"] = row['suspectsKilled'] + row['suspectsInjured']

suspects_df = df.groupby(df.opened.dt.year)['suspectsTotal'].sum().reset_index()

for ind, row in df.iterrows():
    df.loc[ind, "officersTotal"] = row['officersKilled'] + row['officersInjured']
    
officers_df = df.groupby(df.opened.dt.year)['officersTotal'].sum().reset_index()

s_source = ColumnDataSource(suspects_df)
o_source = ColumnDataSource(officers_df)


#add the hover effect for two different lines
hover = HoverTool(mode='mouse', names=['suspects'],
        tooltips = """
            <div>
                <h3>@opened</h3>
                <div><strong>Suspects injured or killed:</strong> @suspectsTotal</div>
            </div>
        """)

hover1 = HoverTool(mode='mouse', names=['officers'],
        tooltips = """
            <div>
                <h3>@opened</h3>
                <div><strong>Officers injured or killed:</strong> @officersTotal</div>
            </div>
        """)

#add plot
p = figure(
    plot_height=600,
    plot_width=800,
    title="Suspects injured or killed by year",
    x_axis_label="Year",
    y_axis_label="Number of suspects killed or injured",
    toolbar_location=None,
    tools="")

p.line(
    x= 'opened',
    y='suspectsTotal',
    line_width=6,
    color='#009bff',
    source=s_source,
    name='suspects')

p.line(
    x= 'opened',
    y='officersTotal',
    line_width=6,
    color='#212121',
    source=o_source,
    name='officers')

p.circle(
    x= 'opened',
    y='suspectsTotal',
    line_width=8,
    color='#009bff',
    source=s_source)

p.circle(
    x= 'opened',
    y='officersTotal',
    line_width=8,
    color='#212121',
    source=o_source)
    
p.y_range.start = 0
p.add_tools(hover, hover1)


#show the result
show(p)


# Conclusion
1. What areas of South Carolina have higher rates of police shootings?
     - Greenville, Richland and Spartanburg counties have highest rates of police shooting incidents.  
2. Are there certain law enforcement agencies that have higher rates of police shootings?
    - The Greenville Sherrif's office much higher police rate than other agencies in the county.
3. Have incidents of police shootings risen since 2009?
    - The number of suspects killed or injured more than tripled between 2009 and 2011. It has remained near the 2011 level since but reached all time in 2018. 
    - The number of officers killed or injured it's lowest number in 2013, but also had it's number in 2018.
4. Is there a correlation between police shootings and the ethnicity of the suspects?
    - There was a lot of shooting incidents missing ethnic data. Further data collection would be required to accurately determine if there's a correlation or not.
5. Is there a correlation between police shootings and the type of incident the officer is responding to?
    - When officers are responding to domestic offenses and traffic stops, the number of suspects killed or injured is considerably higher than other offenses.